In [1]:
import duckdb
from dotenv import load_dotenv

# Para probar que podamos conectarnos al bucket correctamente

In [2]:
load_dotenv()  # take environment variables from .env.

True

In [3]:
print('Probando desde duckdb:')
res = duckdb.sql(
'''
select *
from read_csv_auto('s3://archiva-apagones/penguins.csv')   
''')
print(res)

Probando desde duckdb:
┌─────────┬───────────┬────────────────┬───────────────┬───────────────────┬─────────────┬─────────┐
│ species │  island   │ bill_length_mm │ bill_depth_mm │ flipper_length_mm │ body_mass_g │   sex   │
│ varchar │  varchar  │     double     │    double     │       int64       │    int64    │ varchar │
├─────────┼───────────┼────────────────┼───────────────┼───────────────────┼─────────────┼─────────┤
│ Adelie  │ Torgersen │           39.1 │          18.7 │               181 │        3750 │ MALE    │
│ Adelie  │ Torgersen │           39.5 │          17.4 │               186 │        3800 │ FEMALE  │
│ Adelie  │ Torgersen │           40.3 │          18.0 │               195 │        3250 │ FEMALE  │
│ Adelie  │ Torgersen │           NULL │          NULL │              NULL │        NULL │ NULL    │
│ Adelie  │ Torgersen │           36.7 │          19.3 │               193 │        3450 │ FEMALE  │
│ Adelie  │ Torgersen │           39.3 │          20.6 │            

In [4]:
duckdb.execute(
'''
create or replace table raw_regions_without_service as (
    select *
    from read_json('s3://archiva-apagones/regions_without_service/2023-12-19/*.json', filename=true, auto_detect=true, format='auto') 
)
''')
# select * from read_json('samples/regions_without_service/*.json', filename=true, auto_detect=true, format='auto')

In [5]:
duckdb.sql(
'''
describe
    select
        *
    from raw_regions_without_service
''')

┌─────────────┬────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                          column_type                           │  null   │   key   │ default │  extra  │
│   varchar   │                            varchar                             │ varchar │ varchar │ varchar │ varchar │
├─────────────┼────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ regions     │ STRUCT("name" VARCHAR, percentageClientsWithService DOUBLE, …  │ YES     │ NULL    │ NULL    │ NULL    │
│ timestamp   │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │ NULL    │
│ totals      │ STRUCT(totalClients BIGINT, totalClientsWithService BIGINT, …  │ YES     │ NULL    │ NULL    │ NULL    │
│ filename    │ VARCHAR                                                        │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────────

In [6]:
duckdb.sql(
'''
    select
        strptime("timestamp", '%m/%d/%Y %I:%M %p') as "timestamp",
        totals.totalClients as clientes,
        totals.totalClientsWithService as clientes_con_servicio,
        totals.totalClientsWithoutService as clientes_sin_servicio,
        filename
            .string_split('__')[2]
            .regexp_extract('(.*).json', 1)
            as uhh
    from raw_regions_without_service
''')

┌─────────────────────┬──────────┬───────────────────────┬───────────────────────┬──────────────────────────┐
│      timestamp      │ clientes │ clientes_con_servicio │ clientes_sin_servicio │           uhh            │
│      timestamp      │  int64   │         int64         │         int64         │         varchar          │
├─────────────────────┼──────────┼───────────────────────┼───────────────────────┼──────────────────────────┤
│ 2023-12-19 00:40:00 │  1468223 │               1462627 │                  5596 │ 2023-12-19T00-44-07-0400 │
│ 2023-12-19 00:50:00 │  1468223 │               1457008 │                 11215 │ 2023-12-19T00-54-11-0400 │
│ 2023-12-19 09:55:00 │  1468223 │               1457024 │                 11199 │ 2023-12-19T09-57-57-0400 │
│ 2023-12-19 10:05:00 │  1468223 │               1457020 │                 11203 │ 2023-12-19T10-05-31-0400 │
│ 2023-12-19 10:20:00 │  1468223 │               1457014 │                 11209 │ 2023-12-19T10-23-47-0400 │
│ 2023-12-

In [7]:
duckdb.sql(
'''
describe
    select
        regions[1]
    from raw_regions_without_service
''')

┌─────────────┬────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │                          column_type                           │  null   │   key   │ default │  extra  │
│   varchar   │                            varchar                             │ varchar │ varchar │ varchar │ varchar │
├─────────────┼────────────────────────────────────────────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ regions[1]  │ STRUCT("name" VARCHAR, percentageClientsWithService DOUBLE, …  │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴────────────────────────────────────────────────────────────────┴─────────┴─────────┴─────────┴─────────┘

In [8]:
duckdb.sql(
'''
    select
        regions[1],
        "timestamp",
        totals,
        filename
    from raw_regions_without_service
''')

┌──────────────────────┬─────────────────────┬──────────────────────┬──────────────────────────────────────────────────┐
│      regions[1]      │      timestamp      │        totals        │                     filename                     │
│ struct("name" varc…  │       varchar       │ struct(totalclient…  │                     varchar                      │
├──────────────────────┼─────────────────────┼──────────────────────┼──────────────────────────────────────────────────┤
│ {'name': Arecibo, …  │ 12/19/2023 12:40 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2023 12:50 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2023 09:55 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2023 10:05 AM │ {'totalClients': 1…  │ s3://archiva-apagones/regions_without_service/…  │
│ {'name': Arecibo, …  │ 12/19/2